<div style="text-align: center; line-height: 0; padding-top: 2px;">
  <img src="https://www.quantiaconsulting.com/logos/quantia_logo_orizz.png" alt="Quantia Consulting" style="width: 600px; height: 250px">
</div>

# Concept Drift
---

Dynamic environments are challenging for machine learning methods because data changes over time.

For this example, we will generate a synthetic data stream by concatenating data from 3 different distributions:
- $dist_a$: $\mu=0.4$, $\sigma=0.02$
- $dist_b$: $\mu=0.6$, $\sigma=0.1$
- $dist_c$: $\mu=1.5$, $\sigma=0.05$.

In [1]:
import numpy as np
from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook
from bokeh.layouts import gridplot
from bokeh.palettes import Pastel1
from bokeh.models import Span

In [2]:
def plot_data(dist_a, dist_b, dist_c, drifts=None):
    output_notebook()
    color_0 = Pastel1[3][0]
    color_1 = Pastel1[3][1]
    color_2 = Pastel1[3][2]

    left = figure(plot_width=600, plot_height=400,
                  tools="pan,box_zoom,reset,save",
                  title="drift stream",
                  x_axis_label='samples', y_axis_label='value',
                  background_fill_color="#fafafa"
                  )
    # add some renderers
    left.circle(range(1000), dist_a, legend_label=r"dist_a",
                fill_color=color_0, line_color=color_0, size=4)
    left.circle(range(1000, 2000, 1), dist_b, legend_label=r"dist_b",
                fill_color=color_1, line_color=color_1, size=4)
    left.circle(range(2000, 3000, 1), dist_c, legend_label=r"dist_c",
                fill_color=color_2, line_color=color_2, size=4)

    right = figure(plot_width=300, plot_height=400,
                   tools="pan,box_zoom,reset,save",
                   title="distributions",
                   background_fill_color="#fafafa"
                   )
    hist, edges = np.histogram(dist_a, density=True, bins=50)
    right.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
               fill_color=color_0, line_color=color_0, legend_label='dist_a')
    hist, edges = np.histogram(dist_b, density=True, bins=50)
    right.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
               fill_color=color_1, line_color=color_1, legend_label='dist_b')
    hist, edges = np.histogram(dist_c, density=True, bins=50)
    right.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
               fill_color=color_2, line_color=color_2, legend_label='dist_c')
    
    if drifts is not None:
        for drift_loc in drifts:
            drift_line = Span(location=drift_loc, dimension='height',
                              line_color='red', line_width=2)
            left.add_layout(drift_line)
    
    p = gridplot([[left, right]])
    show(p)

In [3]:
# Generate data for 3 distributions
random_state = np.random.RandomState(seed=42)
dist_a = random_state.normal(0.4, 0.02, 1000)
dist_b = random_state.normal(0.6, 0.1, 1000)
dist_c = random_state.normal(1.5, 0.05, 1000)

# Concatenate data to simulate a data stream with 2 drifts
stream = np.concatenate((dist_a, dist_b, dist_c))

In [4]:
plot_data(dist_a, dist_b, dist_c)

Loading BokehJS ...

As observed above, the synthetic data stream has 3 **abrupt drifts**. An analogy could be trending topics on Twitter which can change quickly.

The goal is to detect that drift has occurred, after samples **1000** and **2000** in the synthetic data stream.

In this example, we will use the [ADaptive WINdowing (`ADWIN`)](https://riverml.xyz/latest/api/drift/ADWIN/) drift detection method.

In [5]:
from river.drift import ADWIN

drift_detector = ADWIN()
drifts = []

for i, val in enumerate(stream):
    drift_detector.update(val)           # Data is processed one sample at a time
    if drift_detector.change_detected:
        print(f'Change detected at index {i}')
        drifts.append(i)
        drift_detector.reset()           # As a best practice, we reset the detector

Change detected at index 1087
Change detected at index 2047


In [6]:
plot_data(dist_a, dist_b, dist_c, drifts)

Loading BokehJS ...

##### ![Quantia Tiny Logo](https://www.quantiaconsulting.com/logos/quantia_logo_tiny.png) Quantia Consulting, srl. All rights reserved.